In [5]:
import hashlib
import os 
import json
from datetime import datetime
from flask import Flask

In [6]:
class Block(object):
    def __init__(self, dictionary):
        # init index, timestamp, data, prev_hash, nonce
        for k, v in dictionary.items():
            setattr(self, k, v)
        
        if not hasattr(self, "nonce"):
            self.nonce = "None"
        if not hasattr(self, "hash"):
            self.hash = self.create_self_hash()

    def header_string(self):
        return str(self.index) + self.prev_hash + self.data + str(self.timestamp) + str(self.nonce)
    
    def create_self_hash(self):
        sha = hashlib.sha256()
        sha.update(self.header_string().encode("utf-8"))
        return sha.hexdigest()
    
    def self_save(self):
        chaindata_dir = "chaindata"
        index_string = str(self.index).zfill(6)
        filename = "%s/%s.json" % (chaindata_dir, index_string)
        with open(filename, "w") as block_file:
            json.dump(self.__dict__(), block_file)

    def __dict__(self):
        info = {}
        info["index"] = str(self.index)
        info["timestamp"] = str(self.timestamp)
        info["prev_hash"] = str(self.prev_hash)
        info["hash"] = str(self.hash)
        info["data"] = str(self.data)
        info["nonce"] = str(self.nonce)
        return info

    def __str__(self):
        return "Block<prev_hash: %s, hash: %s>" % (self.prev_hash, self.hash)

In [7]:
def create_genesis_block():
    block_data = {}
    block_data["index"] = 0
    block_data["timestamp"] = datetime.now()
    block_data["data"] = "Genesis Block Data"
    block_data["prev_hash"] = ""
    block_data["nonce"] = 0
    return Block(block_data)

NUM_ZEROS = 4
def mine(last_block):
    block_data = {}
    block_data["index"] = int(last_block.index) + 1
    block_data["timestamp"] = datetime.now()
    block_data["data"] = "Block #%s" % (int(last_block.index) + 1)
    block_data["prev_hash"] = last_block.hash
    block_data["nonce"] = 0

    tmp_block = Block(block_data)
    while str(tmp_block.hash[0:NUM_ZEROS]) != '0' * NUM_ZEROS:
        block_data["nonce"] += 1
        tmp_block = Block(block_data)

    #dictionary to create the new block object.
    block_data = {}
    block_data['index'] = index
    block_data['prev_hash'] = last_block.hash
    block_data['timestamp'] = timestamp
    block_data['data'] = "Gimme %s dollars" % index
    block_data['hash'] = block_hash
    block_data['nonce'] = nonce
    return Block(block_data)

In [ ]:
def sync():
    node_blocks = []
    # Assuming that the folder and at least initial block exists
    chaindata_dir = 'chaindata'
    if os.path.exists(chaindata_dir):
        for filename in os.listdir(chaindata_dir):
            if filename.endswith('.json'):
                filepath = '%s/%s' % (chaindata_dir, filename)
                with open(filepath, 'r') as block_file:
                    block_info = json.load(block_file)
                    block_object = Block(block_info)
                    node_blocks.append(block_object)
    return node_blocks

In [ ]:
node = Flask(__name__)
@node.route('/blockchain.json', methods=['GET'])
def blockchain_display():
    node_blocks = sync()
    # Convert our blocks into dictionaries
    # so we can send them as json objects later
    python_blocks = []
    for block in node_blocks:
        python_blocks.append(block.__dict__())
        
    json_blocks = json.dumps(python_blocks)
    return json_blocks

In [8]:
if __name__ == "__main__":
    chaindata_dir = "chaindata"
    os.makedirs(chaindata_dir, exist_ok=True)
    if os.listdir(chaindata_dir) == []:
        genesis_block = create_genesis_block()
        genesis_block.self_save()
    
    node.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/May/2022 12:35:56] "GET /blockchain.json HTTP/1.1" 200 -
